<a href="https://colab.research.google.com/github/giulianafernandes/2-machine_learning_and_time_series/blob/main/tech_challenge_manipulando_dados_do_BigQuery_PNAD_Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PNAD-COVID-19 do IBGE**
O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:


- Utilização de no máximo 20 questionamentos realizados na pesquisa;
- Utilizar 3 meses para construção da solução;
- Caracterização dos sintomas clínicos da população;
- Comportamento da população na época da COVID-19;
- Características econômicas da Sociedade;

## Blocos de perguntas do questionário
- Modulo A - Características dos moradores
- Módulo B - COVID-19
- Módulo C - Características de trabalho das pessoas de 14 anos ou mais de idade
- Módulo D - Rendimentos de outras fontes de todos os moradores do domicílio
- Módulo E - Empréstimos
- Módulo F - Domicílio, propriedade e valor do aluguel

#Instalação Spark

In [1]:
# instalando as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
# iniciando o findspark

In [4]:
import findspark
findspark.init()

# Carregando o Spark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .master('local[*]') \
     .appName('Spark') \
     .getOrCreate()

In [6]:
spark

# Carregando o Bigquery no Colab

In [7]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project = 'pos-tech-416222'
location = 'US'
client = bigquery.Client(project = project, location = location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [8]:
job_1 = client.get_job('bquxjob_28046f34_18e2ff6bde2')
df_1 = job_1.to_dataframe()
print(job_1.query)

df_1.head(5)

SELECT ano, 
       mes, 
       sigla_uf,
           CASE 
             WHEN cast(a003 as int64) = 1 THEN 'masculino'
             WHEN cast(a003 as int64) = 2 THEN 'feminino'
             ELSE 'outros'
             END as sexo_biologico,
           CASE 
             WHEN cast(b0011 as int64) = 1 THEN 'sim'
             WHEN cast(b0011 as int64) = 2 THEN 'não'
             ELSE 'não sabe'
             END as febre,
           CASE 
             WHEN cast(b0012 as int64) = 1 THEN 'sim'
             WHEN cast(b0012 as int64) = 2 THEN 'não'
             ELSE 'não sabe'
             END as tosse,
           CASE
             WHEN cast(b0013 as int64) = 1 THEN 'sim'
             WHEN cast(b0013 as int64) = 2 THEN 'não'
             ELSE 'não sabe'
             END as dor_de_garganta,
           CASE
             WHEN cast(b0014 as int64) = 1 THEN 'sim'
             WHEN cast(b0014 as int64) = 2 THEN 'não'
             ELSE 'não sabe'
             END as dificuldade_para_respirar,
        

,ano,mes,sigla_uf,sexo_biologico,febre,tosse,dor_de_garganta,dificuldade_para_respirar,dor_de_cabeca,dor_no_peito,...,perda_de_cheiro_ou_sabor,dor_muscular,diarreia,foi_internado,sedacao_intubacao_ou_respiracao_artificial,tem_plano_de_saude,fez_teste_de_covid,teste_tipo_SWAB,resultado_do_teste,tipo_de_isolamento
0,2020,9,AP,feminino,não,não,não,não,não,não,...,não,não,não,não foi atendido,None,não,não,None,None,"reduziu o contato com as pessoas, porém contin..."
1,2020,9,AP,feminino,não,não,não,não,não,não,...,não,não,não,não foi atendido,None,não,sim,não,None,"reduziu o contato com as pessoas, porém contin..."
2,2020,9,AP,masculino,não,não,não,não,não,não,...,não,não,não,não foi atendido,None,não,não,None,None,None
3,2020,9,AP,feminino,não,não,não,não,não,não,...,não,não,não,não foi atendido,None,não,não,None,None,"reduziu o contato com as pessoas, porém contin..."
4,2020,9,AP,masculino,não,não,não,não,não,não,...,não,não,não,não foi atendido,None,não,não,None,None,não fez restrição alguma


In [9]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149197 entries, 0 to 1149196
Data columns (total 24 columns):
 #   Column                                      Non-Null Count    Dtype 
---  ------                                      --------------    ----- 
 0   ano                                         1149197 non-null  Int64 
 1   mes                                         1149197 non-null  Int64 
 2   sigla_uf                                    1149197 non-null  object
 3   sexo_biologico                              1149197 non-null  object
 4   febre                                       1149197 non-null  object
 5   tosse                                       1149197 non-null  object
 6   dor_de_garganta                             1149197 non-null  object
 7   dificuldade_para_respirar                   1149197 non-null  object
 8   dor_de_cabeca                               1149197 non-null  object
 9   dor_no_peito                                1149197 non-null  object

In [26]:
import numpy as np
import pandas as pd

In [11]:
# Habilitando o apache Arrow
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [12]:
df_sintomas = spark.createDataFrame(df_1)

/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:431: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [14]:
df_sintomas.printSchema()

root
 |-- ano: long (nullable = true)
 |-- mes: long (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- sexo_biologico: string (nullable = true)
 |-- febre: string (nullable = true)
 |-- tosse: string (nullable = true)
 |-- dor_de_garganta: string (nullable = true)
 |-- dificuldade_para_respirar: string (nullable = true)
 |-- dor_de_cabeca: string (nullable = true)
 |-- dor_no_peito: string (nullable = true)
 |-- nausea_enjoo: string (nullable = true)
 |-- nariz_entupido_ou_escorrendo: string (nullable = true)
 |-- fadiga_cansaco: string (nullable = true)
 |-- dor_nos_olhos: string (nullable = true)
 |-- perda_de_cheiro_ou_sabor: string (nullable = true)
 |-- dor_muscular: string (nullable = true)
 |-- diarreia: string (nullable = true)
 |-- foi_internado: string (nullable = true)
 |-- sedacao_intubacao_ou_respiracao_artificial: string (nullable = true)
 |-- tem_plano_de_saude: string (nullable = true)
 |-- fez_teste_de_covid: string (nullable = true)
 |-- teste_tipo_SWAB: 

In [16]:
df_sintomas.show(3)

+----+---+--------+--------------+-----+-----+---------------+-------------------------+-------------+------------+------------+----------------------------+--------------+-------------+------------------------+------------+--------+----------------+------------------------------------------+------------------+------------------+---------------+------------------+--------------------+
| ano|mes|sigla_uf|sexo_biologico|febre|tosse|dor_de_garganta|dificuldade_para_respirar|dor_de_cabeca|dor_no_peito|nausea_enjoo|nariz_entupido_ou_escorrendo|fadiga_cansaco|dor_nos_olhos|perda_de_cheiro_ou_sabor|dor_muscular|diarreia|   foi_internado|sedacao_intubacao_ou_respiracao_artificial|tem_plano_de_saude|fez_teste_de_covid|teste_tipo_SWAB|resultado_do_teste|  tipo_de_isolamento|
+----+---+--------+--------------+-----+-----+---------------+-------------------------+-------------+------------+------------+----------------------------+--------------+-------------+------------------------+------------+

In [17]:
# carregando o segundo dataframe - características do morador

In [18]:
job_2 = client.get_job('bquxjob_2a85e121_18e34c228de')
df_2 = job_2.to_dataframe()
print(job_2.query)

df_2.head(5)

SELECT ano, 
       mes,
       sigla_uf,
       a002 as idade,
       CASE 
          WHEN cast(a003 as int64) = 1 THEN 'masculino'
          WHEN cast(a003 as int64) = 2 THEN 'feminino'
          ELSE 'outros'
          END as sexo_biologico,
       CASE 
          WHEN cast(a004 as int64) = 1 THEN 'branca'
          WHEN cast(a004 as int64) = 2 THEN 'preta'
          WHEN cast(a004 as int64) = 3 THEN 'amarela'
          WHEN cast(a004 as int64) = 4 THEN 'parda'
          WHEN cast(a004 as int64) = 4 THEN 'indígena'
          END as cor_ou_raca,
       CASE 
          WHEN cast(a005 as int64) = 1 THEN 'sem instução'
          WHEN cast(a005 as int64) = 2 THEN 'fundamental incompleto'
          WHEN cast(a005 as int64) = 3 THEN 'fundamental completo'
          WHEN cast(a005 as int64) = 4 THEN 'medio incompleto'
          WHEN cast(a005 as int64) = 5 THEN 'medio completo'
          WHEN cast(a005 as int64) = 6 THEN 'superior incompleto'
          WHEN cast(a005 as int64) = 7 THEN 'sup

,ano,mes,sigla_uf,idade,sexo_biologico,cor_ou_raca,escolaridade,esta_matriculado_atualmente,ensino_publico_ou_privado,tem_aulas_presenciais,foram_disponibilizadas_atividades_escolares_para_realizar_em_casa,porque_nao_realizou_as_atividades_escolares
0,2020,9,AP,80,masculino,parda,medio completo,None,None,None,None,None
1,2020,9,AP,83,feminino,parda,medio completo,None,None,None,None,None
2,2020,9,AP,81,feminino,parda,sem instução,None,None,None,None,None
3,2020,9,AP,88,feminino,parda,sem instução,None,None,None,None,None
4,2020,9,AP,87,feminino,parda,sem instução,None,None,None,None,None


In [19]:
df_caracteristicas = spark.createDataFrame(df_2)
df_caracteristicas.show(3)

/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:431: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  arrow_data = [[(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]
/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Cannot specify a mask or a size when passing an object that is converted with the __arrow_array__ protocol.
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
/content/spark-3.1.2-bin-hadoop2.7/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+----+---+--------+-----+--------------+-----------+--------------+---------------------------+-------------------------+---------------------+-----------------------------------------------------------------+-------------------------------------------+
| ano|mes|sigla_uf|idade|sexo_biologico|cor_ou_raca|  escolaridade|esta_matriculado_atualmente|ensino_publico_ou_privado|tem_aulas_presenciais|foram_disponibilizadas_atividades_escolares_para_realizar_em_casa|porque_nao_realizou_as_atividades_escolares|
+----+---+--------+-----+--------------+-----------+--------------+---------------------------+-------------------------+---------------------+-----------------------------------------------------------------+-------------------------------------------+
|2020|  9|      AP|   80|     masculino|      parda|medio completo|                       null|                     null|                 null|                                                             null|                             

In [22]:
df_caracteristicas.printSchema()

root
 |-- ano: long (nullable = true)
 |-- mes: long (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- sexo_biologico: string (nullable = true)
 |-- cor_ou_raca: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- esta_matriculado_atualmente: string (nullable = true)
 |-- ensino_publico_ou_privado: string (nullable = true)
 |-- tem_aulas_presenciais: string (nullable = true)
 |-- foram_disponibilizadas_atividades_escolares_para_realizar_em_casa: string (nullable = true)
 |-- porque_nao_realizou_as_atividades_escolares: string (nullable = true)



In [69]:
df_caracteristicas\
            .select('sigla_uf', 'esta_matriculado_atualmente', 'ensino_publico_ou_privado')\
            .filter(df_caracteristicas.esta_matriculado_atualmente.startswith("sim"))\
            .filter(df_caracteristicas.ensino_publico_ou_privado.isNotNull())\
            .show(20, False)

+--------+---------------------------+-------------------------+
|sigla_uf|esta_matriculado_atualmente|ensino_publico_ou_privado|
+--------+---------------------------+-------------------------+
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|AP      |sim                        |pública                  |
|TO      |sim                        |pública                  |
|TO      |sim                        |privada                  |
|TO      |sim            